In [2]:
#importing libraries
import sqlite3
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, Column, Integer, Float, String, Table, MetaData
from sqlalchemy.orm import declarative_base, sessionmaker
from bokeh.plotting import figure, show
from bokeh.layouts import gridplot
from bokeh.io import output_notebook
import pandas as pd

In [3]:
# Define database models
Base = declarative_base()

class TrainingData(Base):
    """Table to store training data."""
    __tablename__ = 'training_data'
    x = Column(Float, primary_key=True)
    y1 = Column(Float)
    y2 = Column(Float)
    y3 = Column(Float)
    y4 = Column(Float)

In [4]:
class IdealFunctions(Base):
    """Table to store ideal functions."""
    __tablename__ = 'ideal_functions'
    x = Column(Float, primary_key=True)
    y1 = Column(Float)
    y2 = Column(Float)
    y3 = Column(Float)
    y4 = Column(Float)
    y5 = Column(Float)
    y6 = Column(Float)
    y7 = Column(Float)
    y8 = Column(Float)
    y9 = Column(Float)
    y10 = Column(Float)
    y11 = Column(Float)
    y12 = Column(Float)
    y13 = Column(Float)
    y14 = Column(Float)
    y15 = Column(Float)
    y16 = Column(Float)
    y17 = Column(Float)
    y18 = Column(Float)
    y19 = Column(Float)
    y20 = Column(Float)
    y21 = Column(Float)
    y22 = Column(Float)
    y23 = Column(Float)
    y24 = Column(Float)
    y25 = Column(Float)
    y26 = Column(Float)
    y27 = Column(Float)
    y28 = Column(Float)
    y29 = Column(Float)
    y30 = Column(Float)
    y31 = Column(Float)
    y32 = Column(Float)
    y33 = Column(Float)
    y34 = Column(Float)
    y35 = Column(Float)
    y36 = Column(Float)
    y37 = Column(Float)
    y38 = Column(Float)
    y39 = Column(Float)
    y40 = Column(Float)
    y41 = Column(Float)
    y42 = Column(Float)
    y43 = Column(Float)
    y44 = Column(Float)
    y45 = Column(Float)
    y46 = Column(Float)
    y47 = Column(Float)
    y48 = Column(Float)
    y49 = Column(Float)
    y50 = Column(Float)

In [5]:
class TestDataMapping(Base):
    """Table to store test data mappings."""
    __tablename__ = 'test_data_mapping'
    id = Column(Integer, primary_key=True, autoincrement=True)
    x = Column(Float)
    y = Column(Float)
    delta_y = Column(Float)
    ideal_function_no = Column(String)

In [6]:
class DataLoader:
    """Handles loading and storing data into the SQLite database."""
    def __init__(self, db_url):
        self.engine = create_engine(db_url)
        Base.metadata.create_all(self.engine)
        self.Session = sessionmaker(bind=self.engine)

    def load_csv_to_table(self, file_path, table_class):
        df = pd.read_csv(file_path)
        with self.Session() as session:
            session.bulk_insert_mappings(table_class, df.to_dict(orient='records'))
            session.commit()

    def load_training_data(self, train_csv):
        self.load_csv_to_table(train_csv, TrainingData)

    def load_ideal_functions(self, ideal_csv):
        self.load_csv_to_table(ideal_csv, IdealFunctions)

    def load_test_data(self, test_csv):
        df = pd.read_csv(test_csv)
        with self.Session() as session:
            session.bulk_insert_mappings(TestDataMapping, df.to_dict(orient='records'))
            session.commit()


In [7]:
#selecting the ideal functin
class IdealFunctionSelector:
    """Handles selection of ideal functions using least square deviation."""
    def __init__(self, session):
        self.session = session

    def select_ideal_functions(self, training_data, ideal_functions):
        """Finds the ideal functions minimizing the squared deviations for each training column."""
        ideal_map = {}
        for col in training_data.columns[1:]:  # Iterate through Y1, Y2, Y3, Y4
            best_fit = None
            min_deviation = float('inf')
            for function in ideal_functions.columns[1:]:  # Iterate through ideal functions Y1 to Y50
                deviation = ((training_data[col] - ideal_functions[function]) ** 2).sum()
                if deviation < min_deviation:
                    min_deviation = deviation
                    best_fit = function
            ideal_map[col] = best_fit
        return ideal_map

In [8]:
#mapping the data
class TestDataMapper:
    """Maps test data to the chosen ideal functions."""
    def __init__(self, session, ideal_map, max_deviation_thresholds):
        self.session = session
        self.ideal_map = ideal_map
        self.max_deviation_thresholds = max_deviation_thresholds

    def map_test_data(self, test_data, ideal_functions):
        """Maps test data points to ideal functions based on deviation criteria."""
        mappings = []
        for _, row in test_data.iterrows():
            x, y = row['x'], row['y']
            best_fit = None
            min_deviation = float('inf')

            for train_col, ideal_func in self.ideal_map.items():
                ideal_y = ideal_functions.loc[ideal_functions['x'] == x, ideal_func].values[0]
                deviation = abs(y - ideal_y)
                if deviation <= self.max_deviation_thresholds[train_col] and deviation < min_deviation:
                    min_deviation = deviation
                    best_fit = ideal_func

            if best_fit:
                mappings.append({'x': x, 'y': y, 'delta_y': min_deviation, 'ideal_function_no': best_fit})
        return mappings

In [9]:
#for data visualization
class Visualization:
    """Handles visualization of the data."""
    def __init__(self):
        output_notebook()  # Ensures plots display in Jupyter Notebook environments

    def plot_data(self, training_data, ideal_functions, test_data_mappings, ideal_map):
        """
        Visualizes the training data, ideal functions, and test data mappings.
        Args:
            training_data (pd.DataFrame): Training data with x and y1, y2, etc.
            ideal_functions (pd.DataFrame): Ideal functions with x and y1, y2, etc.
            test_data_mappings (list): List of mappings with x, y, delta_y, and ideal_function_no.
            ideal_map (dict): Mapping of training columns to ideal function columns.
        """
        plots = []

        for col in training_data.columns[1:]:  # Iterate through Y1, Y2, Y3, etc.
            # Create a figure for each training data column
            p = figure(
                title=f'Training Data vs Ideal Function: {col}',
                x_axis_label='x', y_axis_label='y',
                width=600, height=400
            )

            # Scatter plot for training data
            p.scatter(training_data['x'], training_data[col], color="blue", legend_label=f"Training {col}")

            # Line plot for the ideal function
            ideal_col = ideal_map[col]
            p.line(ideal_functions['x'], ideal_functions[ideal_col], color="red", legend_label=f"Ideal {ideal_col}")

            # Scatter plot for test data mappings
            for mapping in test_data_mappings:
                if mapping['ideal_function_no'] == ideal_col:
                    p.scatter(
                        [mapping['x']],
                        [mapping['y']],
                        color="green", size=8,
                        legend_label="Mapped Test Data", marker="circle"
                    )

            p.legend.location = "top_left"
            plots.append(p)

        # Display all plots in a grid layout
        grid = gridplot([plots])
        show(grid)

In [10]:
#run to use
if __name__ == "__main__":
    db_url = 'sqlite:///functions.db'
    loader = DataLoader(db_url)

    # Load data into the database(file path is local)
    training_csv = "/content/train.csv"
    ideal_csv = "/content/ideal.csv"
    test_csv = "/content/test.csv"

    loader.load_training_data(training_csv)
    loader.load_ideal_functions(ideal_csv)
    loader.load_test_data(test_csv)

    # Load data into DataFrames for processing
    training_data = pd.read_sql("SELECT * FROM training_data", loader.engine)
    ideal_functions = pd.read_sql("SELECT * FROM ideal_functions", loader.engine)
    test_data = pd.read_sql("SELECT * FROM test_data_mapping", loader.engine)

    # Select ideal functions
    selector = IdealFunctionSelector(loader.Session())
    ideal_map = selector.select_ideal_functions(training_data, ideal_functions)

    # Compute max deviation thresholds for each training column
    max_deviation_thresholds = {}
    for train_col, ideal_func in ideal_map.items():
        max_deviation = ((training_data[train_col] - ideal_functions[ideal_func]) ** 2).max()
        max_deviation_thresholds[train_col] = np.sqrt(2) * np.sqrt(max_deviation)

    # Map test data
    mapper = TestDataMapper(loader.Session(), ideal_map, max_deviation_thresholds)
    mappings = mapper.map_test_data(test_data, ideal_functions)

    # Save mappings to database
    mapped_df = pd.DataFrame(mappings)
    mapped_df.to_sql("test_data_mapping", loader.engine, if_exists="replace", index=False)

    # Visualize data
    # viz = Visualization()
    # viz.plot_data(training_data, ideal_functions, mappings, ideal_map)
    viz = Visualization()
    viz.plot_data(training_data, ideal_functions, mappings, ideal_map)

In [ ]:
#database showing the results and mapped function
from google.colab import files
files.download('functions.db')